Q&A Assistant Agent by Shrayajyoti Dey

In [ ]:
!pip install --upgrade transformers --quiet

In [ ]:
from transformers import pipeline
import re
import datetime
import operator
import time

# Initialize the LLM pipeline using PyTorch
llm = pipeline(
    "text2text-generation",
    model="google/flan-t5-large",
    max_length=128,
    framework="pt"
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [ ]:

# Sample questions for testing
test_questions = [
    {"input": "What is 23 + 19?", "expected_tool": "TOOL_CALCULATOR"},
    {"input": "Can you tell me the current time?", "expected_tool": "TOOL_TIME"},
    {"input": "What time zone am I in?", "expected_tool": "TOOL_TIMEZONE"},
    {"input": "What is today's date and day?", "expected_tool": "TOOL_DATE"},
    {"input": "Can you debug this python code: for i in range(5) print(i)", "expected_tool": "TOOL_DEBUG"},
    {"input": "Who is the president of the USA?", "expected_tool": "ANSWER"},
    {"input": "Calculate 10 * 5 - 3", "expected_tool": "TOOL_CALCULATOR"},
    {"input": "What is an AI?", "expected_tool": "ANSWER"},
]

# Function to run tests
def run_tests():
    print("\n--- Running automated tests ---")
    for i, q in enumerate(test_questions):
        user_input = q["input"]
        expected = q["expected_tool"]
        print(f"\nTest {i+1}: {user_input}")

        decision = llm(f"""
You are an AI agent with access to tools:
calculator, time checker, timezone checker, date checker, python debugger.

Rules:
1. If the user input contains math, respond exactly: TOOL_CALCULATOR
2. If the user input asks for the current time, respond exactly: TOOL_TIME
3. If the user input asks for time zone, respond exactly: TOOL_TIMEZONE
4. If the user input asks for today's date or day, respond exactly: TOOL_DATE
5. If the user input asks to debug or fix Python code, respond exactly: TOOL_DEBUG
6. For all other questions, respond exactly: ANSWER

Do not explain anything.
Only output one of these:
TOOL_CALCULATOR, TOOL_TIME, TOOL_TIMEZONE, TOOL_DATE, TOOL_DEBUG, ANSWER.

User Input: {user_input}
Agent Decision:
""")[0]["generated_text"].strip()

        print("Expected tool:", expected)
        print("Model decision:", decision)

        # Run the selected tool
        if decision == "TOOL_CALCULATOR":
            result = calculator_tool(user_input)
        elif decision == "TOOL_TIME":
            result = time_tool()
        elif decision == "TOOL_TIMEZONE":
            result = timezone_tool()
        elif decision == "TOOL_DATE":
            result = date_day_tool()
        elif decision == "TOOL_DEBUG":
            result = debug_tool(user_input)
        else:
            result = chat_tool(user_input)

        print("Model response:", result)

    print("\n--- Tests complete ---")


In [ ]:

print("Agentic AI (type 'exit' to quit)")
print("To run test cases type 'run_tests'")

# ------------------ TOOLS ------------------

# Safe calculator
ops = {'+': operator.add, '-': operator.sub, '*': operator.mul, '/': operator.truediv}

def calculator_tool(text):
    try:
        numbers = [int(n) for n in re.findall(r"\d+", text)]
        operators = re.findall(r"[\+\-\*/]", text)
        result = numbers[0]
        for i, op in enumerate(operators):
            result = ops[op](result, numbers[i+1])
        return str(result)
    except:
        return "I couldn't calculate that."

def time_tool():
    return datetime.datetime.now().strftime("%H:%M:%S")

def timezone_tool():
    tz_name = time.tzname[0]
    offset_hours = -time.timezone // 3600
    return f"Current time zone: {tz_name} (UTC{offset_hours:+d})"

def date_day_tool():
    today = datetime.datetime.now()
    return today.strftime("Today's date is %Y-%m-%d and today is %A")

def debug_tool(code_text):
    prompt = f"""
You are a Python debugging assistant.
The user gives a simple Python program.
Explain possible errors and how to fix them.
Do NOT run the code.
Keep the explanation simple.

Code:
{code_text}

Debugging advice:
"""
    return llm(prompt)[0]["generated_text"].strip()

def chat_tool(user_input):
    prompt_chat = f"""
You are a helpful AI assistant.
Answer clearly in simple language.
User: {user_input}
Assistant:
"""
    return llm(prompt_chat)[0]["generated_text"].strip()

# ------------------ AGENT ------------------

def decide_and_act(user_input):
    prompt = f"""
You are an AI agent with access to tools:
calculator, time checker, timezone checker, date checker, python debugger.

Rules:
1. If the user input contains math, respond exactly: TOOL_CALCULATOR
2. If the user input asks for the current time, respond exactly: TOOL_TIME
3. If the user input asks for time zone, respond exactly: TOOL_TIMEZONE
4. If the user input asks for today's date or day, respond exactly: TOOL_DATE
5. If the user input asks to debug or fix Python code, respond exactly: TOOL_DEBUG
6. For all other questions, respond exactly: ANSWER

Do not explain anything.
Only output one of these:
TOOL_CALCULATOR, TOOL_TIME, TOOL_TIMEZONE, TOOL_DATE, TOOL_DEBUG, ANSWER.

User Input: {user_input}
Agent Decision:
"""
    decision = llm(prompt)[0]["generated_text"].strip()

    if decision == "TOOL_CALCULATOR":
        return calculator_tool(user_input)
    elif decision == "TOOL_TIME":
        return time_tool()
    elif decision == "TOOL_TIMEZONE":
        return timezone_tool()
    elif decision == "TOOL_DATE":
        return date_day_tool()
    elif decision == "TOOL_DEBUG":
        return debug_tool(user_input)
    else:
        return chat_tool(user_input)

# ------------------ MAIN LOOP ------------------

while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        print("Agent: Goodbye!")
        break
    elif user_input.lower() == "run_tests":  # ✅ FIXED HERE
        try:
            run_tests()
        except NameError:
            print("No test cases found! Please define `run_tests()` in a previous cell.")
        continue

    result = decide_and_act(user_input)
    print("Agent:", result)


Agentic AI (type 'exit' to quit)
To run test cases type 'run_tests'
You: run_tests

--- Running automated tests ---

Test 1: What is 23 + 19?
Expected tool: TOOL_CALCULATOR
Model decision: TOOL_CALCULATOR
Model response: 42

Test 2: Can you tell me the current time?
Expected tool: TOOL_TIME
Model decision: TOOL_TIME
Model response: 14:18:05

Test 3: What time zone am I in?
Expected tool: TOOL_TIMEZONE
Model decision: TOOL_TIMEZONE
Model response: Current time zone: UTC (UTC+0)

Test 4: What is today's date and day?
Expected tool: TOOL_DATE
Model decision: TOOL_DATE
Model response: Today's date is 2025-12-25 and today is Thursday

Test 5: Can you debug this python code: for i in range(5) print(i)
Expected tool: TOOL_DEBUG
Model decision: TOOL_DEBUG
Model response: for i in range(5): print(i)

Test 6: Who is the president of the USA?
Expected tool: ANSWER
Model decision: ANSWER
Model response: The president of the USA is Donald Trump.

Test 7: Calculate 10 * 5 - 3
Expected tool: TOOL_CAL